<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ラベルなしデータでの事前学習
- LLMが訓練中に生成したテキストの品質を評価するために、訓練データと検証データセットでの損失（誤差）を計算する。
- 訓練関数を実装し、LLMの事前学習を行う。
- LLMを引き続き訓練するための重みを保存し、LLMに読み込む。
- OpenAIから事前学習済みの重みを読み込む。

## 5.1 生成テキストモデルを評価する

In [ ]:
# GPTモデルの設定
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 語彙のサイズ
    "context_length": 256, # コンテキストの長さ
    "emb_dim": 768,         # 埋め込み次元数
    "n_heads": 12,          # Attentionヘッドの数
    "n_layers": 12,         # 層の数
    "drop_rate": 0.1,       # ドロップアウト率
    "qkv_bias": False       # クエリ、キー、バリューの計算にバイアスを使用するか
}

In [ ]:
import tiktoken
import torch
import torch.nn as nn

tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
# Layer Normalizationの実装と適用をモジュール化

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False) # unbiased=True 有偏分散
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # 標準化（ゼロ除算にならないようにepsを加える）
        return self.scale * norm_x + self.shift

# GELU活性化関数の実装
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

# フォードフォワードニューラルネットワークモジュール
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), # 1つ目の線形層で入力の次元から4倍にして出力
            GELU(), # GELU活性化関数
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), # 2つ目の線形層で4倍にした入力の次元数を4分の1の元の次元数に戻して出力
        )

    def forward(self, x):
        return self.layers(x)

# 3章のMulti-head Attentionコンポーネント
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0) # 出力に次元数がヘッド数と割り切れない場合は警告出す

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 出力次元数をヘッド数で分割

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out) # Linear層を使ってヘッドの出力を組み合わせる
        self.dropout = nn.Dropout(dropout)
        # maskを作成 (context_lengthの形状に従う)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)

        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape # テンソルの形状は(batch, num_tokens, d_out)

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # num_heads次元を追加して行列えお暗黙的に分割
        # 最後の次元を展開し、形状を(batch, num_tokens, d_out) -> (batch, num_tokens, num_heads, head_dim)に変換
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 形状を(batch, num_tokens, num_heads, head_dim) ->
        # (batch, num_heads, num_tokens, head_dim)に変換
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3) # 各ヘッドのドット積を計算

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # マスクをトークン数で切り捨て

        attn_scores.masked_fill_(mask_bool, -torch.inf) # Attentionスコアを埋めるためにマスクを使う

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) # テンソルの形状は(batch, num_tokens, n_heads, head_dim)
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)

        context_vec = self.out_proj(context_vec) # 線形射影を追加

        return context_vec

# GPTアーキテクチャのTransformerブロックコンポーネント
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.attn = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        shortcut = x # Attentionブロックのショートカット接続(入力のオリジナル)
        x = self.norm1(x)
        x = self.attn(x)
        x = self.drop_shortcut(x)
        x = x + shortcut # 元の入力をショートカット接続として入力

        shortcut = x # フィードフォワードブロックのショートカット接続(前段の処理結果)
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

# モデルアーキテクチャの実装
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Transformerブロック
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)

        # デバイス設定
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();

コンテキストの長さ（context_length）を1024->256に短縮したことで、モデルの訓練に必要な計算量が少なくなり、標準的なラップトップコンピュータでも計算可能になる

In [ ]:
# テキストをトークンIDに変換するユーティリティ関数

# テキストを生成するGPTモデルの関数
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:] # サポートされているコンテキストサイズを超える場合は現在のコンテキストを切り詰める
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]

        idx_next = torch.argmax(logits, dim=-1, keepdim=True)

        idx = torch.cat((idx, idx_next), dim=1)

    return idx

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # unsqueeze(0)はバッチ次元を追加する
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # バッチ次元を削除
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print(f"Output text:\n {token_ids_to_text(token_ids, tokenizer)}")

In [ ]:
# 損失関数の実装

In [ ]:
# 2つの入力サンプルについて考える
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

# モデルに生成させたいトークンID
# inputsを右に1つシフトさせたもの
targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [ ]:
# ロジットベクトルを計算（確率スコア）
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # 語彙の各トークンの確率
print(probas.shape)

1つ目の数字`2`は、入力の2つのサンプル（行）に対応している。バッチサイズともいう。

2つめの数字`3`は、各入力（列）のトークン数。

最後の数字は埋め込み次元数であり、語彙のサイズによって決定される。

In [ ]:
# 確率スコアにargmax()関数を適用して対応するトークンIDを取得
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print(f"Token IDs:\n{token_ids}")

In [ ]:
# トークンIDをテキストに戻す
print(f"Tragets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

In [ ]:
# 2つの入力テキストそれぞれについて、ターゲットトークンに対応するソフトマックス確率スコアを出力
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print(f"Text 1: {target_probas_1}")

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print(f"Text 2: {target_probas_2}")

LLMの訓練の目的は正しいトークンが生成される確率の最大化

In [ ]:
# 対数確率
# 最適化の観点から、確率スコアをそのまま使用するよりも対数にした方が扱いやすい
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

In [ ]:
# 対数確率の平均を求めて、1つのスコアにまとめる
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

訓練プロセスの一部としてモデルの重みを更新しながら、この平均対数確率をできるだけ0に近づけることが目標となる。

ディープラーニングでは、平均対数確率を直接0に近づけるのではなく、負の平均対数確率を0に近づけるのが一般的

`負の対数確率`: 単に平均対数確率に`-1`をかけた値のこと

In [ ]:
# 負の対数確率を求める
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

### 交差エントロピー誤差
2つの確率分布の差を定量的に計測する指標であり、機械学習やディープラーニングでよく使われる。モデルが生成したトークン確率に基づく、ターゲットトークンの負の平均対数確率とほぼ同じもの。

In [ ]:
# ロジットテンソルとターゲットテンソルの形状を確認
print(f"Logits shape: {logits.shape}")
print(f"Targets shape: {targets.shape}")

Logitsテンソル -> (バッチサイズ、トークン数、語彙のサイズ)

Targetsテンソル -> (バッチサイズ、トークン数)

cross_entropy()損失関数を使用する場合には、これらのテンソルをバッチ次元で結合することでフラット化( faltten() )しておく必要がある。

In [ ]:
logits_flat = logits.flatten(0, 1)
target_flat = targets.flatten()
print(f"Flattened logits: {logits_flat.shape}")
print(f"Flattened targets: {target_flat.shape}")

In [ ]:
# PyTorchのCrossEntropy関数はこれまでのすべての作業を自動化でやってくれる
# softmax() -> ターゲットIDに対する確率スコア選択 -> 負の平均対数確率計算
loss = torch.nn.functional.cross_entropy(logits_flat, target_flat)
print(loss)

### パープレキシティ
- 言語モデルのようなタスクでモデルの性能評価する際に、交差エントロピー誤差と並んで使われる指標
- シーケンスの次に来るトークンを予測するモデルの不確かさを、より解釈しやすい方法で理解するための手段となる
- モデルが予測した確率分布が、データセット内の単語の実際の分布とどの程度一致するかを計測する
- 損失(誤差)と同様に、パープレキシティが低いほど、モデルの予測が実際の分布に近いことを意味する
- しばしば損失関数よりも解釈がしやすいとみなされるのは、各ステップでモデルが不確かとなる実質的な語彙のサイズを示すからであるパープレキシティが`tensor(46725.8203)`の場合、「語彙に含まれる48,725個のトークンのうち、次に来るトークンとしてどれを生成すべきかについてモデルは確信を持てない」と解釈できる

In [ ]:
# 訓練データと検証データで損失を計算する
# 訓練用、検証用データの準備
import os
import requests

file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    text_data = response.text
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)

else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()

In [ ]:
# データセット内の文字数とトークン数を確認
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print(f"Characters: {total_characters}")
print(f"Tokens: {total_tokens}")

LLMを可変長の入力で訓練するのも効果的 -> 様々な種類の入力にLLMを上手く汎用化させるのに役立つ

In [ ]:
# データのシャッフルと読み込みの実装
train_ration = 0.90
split_idx = int(train_ration * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

In [ ]:
# バッチ入力と目的変数のためのデータセット
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # トークナイザーで全テキストをトークン化
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max length+1"

        # スライディングウィンドウを使ってmax_lengthの長さのシーケンスに分割
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
# 入力変数と目的変数のペアでバッチを生成するデータローダー
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                      stride=128, shuffle=True, drop_last=True,
                      num_workers=0):
    # トークナイザーを初期化
    tokenizer = tiktoken.get_encoding("gpt2")

    # データセット作成
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, # drop_last=Trueは、指定されたbatch_sizeよりも最後のバッチが短い場合に、訓練中の損失値のスパイクを防ぐためにそのバッチを除外
        num_workers=num_workers # 前処理に使用するCPU数
    )

    return dataloader

In [ ]:
# 訓練データセットと検証データセットのデータローダーを作成
torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
# データローダーが問題なく作成されたか確認
print("Train loader: ")
for x, y in train_loader:
    print(x.shape, y.shape)

print("Validation loader: ")
for x, y in val_loader:
    print(x.shape, y.shape)

In [ ]:
# 交差エントロピー誤差を計算するユーティリティ関数を実装
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
        logits.flatten(0, 1), target_batch.flatten()
    )

    return loss


In [ ]:
# 訓練と検証の損失を計算する関数
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        # num_batchesが指定されていない場合
        # すべてのバッチを反復処理
        num_batches = len(data_loader)
    else:
        # num_batchesがデータローダーのバッチ数を超えている場合、データローダーのバッチ数と一致するように調節
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(
                input_batch, target_batch, model, device
            )
            total_loss += loss.item() # 各バッチの損失を計算
        else:
            break

    return total_loss / num_batches # 全バッチの損失の平均を計算 (バッチ総数に対する損失の平均値)

In [ ]:
# calc_loss_loader()を訓練データセットと検証データセットに適用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device) # GPU搭載であればLLMの訓練をGPUで行う

with torch.no_grad():
    # まだ訓練していないため、効率化のために勾配の追跡を無効にする
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print(f"Training loss: {train_loss}")
print(f"Validation loss: {val_loss}")

損失が大きいのはLLMがまだ訓練を行っていないから

訓練データセットと検証データセットに次に現れるトークンを生成するようにモデルを訓練した場合、損失値は0に近づいていく

## LLMを訓練する

In [ ]:
# LLMの事前学習を行うためのメイン関数
def train_model_simple(model, train_loader, val_loader, optimizer, device,
                       num_epochs, eval_freq, eval_iter, start_context, tokenizer):
    train_losses, val_losses, track_tokens_seen = [], [], [] # 損失と既視のトークンを追跡するためにリストを初期化
    tokens_seen, global_step = 0, -1

    for epoch in range(num_epochs):
        model.train() # 学習モード

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # 前回のバッチ反復処理で計算された損失の勾配をリセット
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # 損失勾配を計算
            optimizer.step() # 損失の勾配を使ってモデルの重みを更新
            tokens_seen += input_batch.numel()

            global_step += 1

            if global_step % eval_freq == 0: # オプションの評価ステップ
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter
                )
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): Train loss {train_loss:.3f} Val loss {val_loss:.3f}")

        # 各エポックの後にサンプルテキストを出力
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    # モデルを更新するたびに訓練データセットと検証データセットでの損失を出力
    model.eval() # 評価モード
    with torch.no_grad():
        train_loss = calc_loss_loader(
            train_loader, model, device, num_batches=eval_iter
        )
        val_loss = calc_loss_loader(
            val_loader, model, device, num_batches=eval_iter
        )

    model.train()
    return train_loss, val_loss

def generate_and_print_sample(model, tokenizer, device, start_context):
    # 訓練中にモデルが改善されたかどうかを追跡する関数
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded, max_new_tokens=50, context_size=context_size
        )

    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " ")) # コンパクトな出力フォーマット
    model.train()

### AdamW
Adamオプティマイザの改良版

大きな重みにペナルティを課すことでモデルの複雑さを最小限に抑え、過剰適合を防ぐことを目的として、重みの減衰の処理方法が改善されている。この調整によってより効率的な正則化が可能になり、モデルの汎用性能が向上することから、AdamWはLLMの訓練に非常によく使われている。

In [ ]:
# GPTModelインスタンスを10エポックにわたって訓練
torch.manual_seed(123)

model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.0004, weight_decay=0.1 # 重み減衰
)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

In [ ]:
from numpy import integer
from matplotlib.lines import lineStyles
# 損失についてプロット
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(5, 3))
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))

    ax2 = ax1.twiny() # 同じy軸を共有する2つ目のx軸を作成
    ax2.plot(tokens_seen, train_losses, alpha=0) # メモリを揃えるための不可視プロット
    ax2.set_xlabel("Tokens seen")
    fig.tight_layout()
    plt.show()

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

モデルがデータに対して過剰適合している

モデルが訓練データの文章をそのまま記憶していることがわかる -> 非常に小さい訓練データセットを使用して複数回エポックを回しているため

本来はもっと大きなデータセットで1エポックだけ訓練するのが一般的

## 5.3 ランダム性をコントロールするデコーディング戦略

In [ ]:
model.to("cpu")
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)
print(f"Output text:\n {token_ids_to_text(token_ids, tokenizer)}")

In [ ]:
# Temperature Scalling
# 確率的サンプリング
vocab = {
    "closer": 0,
    "every": 1,
    "effort": 2,
    "forward": 3,
    "inches": 4,
    "moves": 5,
    "pizza": 6,
    "toward": 7,
    "you": 8,
}

inverse_vocab = {v: k for k, v in vocab.items()}

next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

probas = torch.softmax(next_token_logits, dim=0)
next_token_id = torch.argmax(probas).item()

print(inverse_vocab[next_token_id])

In [ ]:
# 確率的サンプリングを実装するために、torch.multinominal()関数に置き換え
torch.manual_seed(123)
next_token_id = torch.multinomial(probas, num_samples=1).item()
print(inverse_vocab[next_token_id])

torch.multinomial()関数は、次に来るトークンをその確率スコアに比例する形でサンプリングする

ただし、確率スコアによるサンプリングのため毎回選択されるわけではない

In [ ]:
# 100回繰り返す
def print_sampled_tokens(probas):
    torch.manual_seed(123)
    sample = [torch.multinomial(probas, num_samples=1).item()
                for i in range(1_000)]
    sampled_ids = torch.bincount(torch.tensor(sample))
    for i, freq in enumerate(sampled_ids):
        print(f"{freq} x {inverse_vocab[i]}")

print_sampled_tokens(probas)

### 温度スケーリング: 0よりも大きい数でロジットを割ることのもったいぶった言い方

In [ ]:
def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    return torch.softmax(scaled_logits, dim=0)

temperatures = [1, 0.1, 5]

scaled_probas = [softmax_with_temperature(next_token_logits, T) for T in temperatures]

In [ ]:
# 異なる温度設定でスケーリングした確率を並べる
x = torch.arange(len(vocab))
bar_width = 0.15

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 3))
for i, T in enumerate(temperatures):
    rects = ax.bar(x + i * bar_width, scaled_probas[i], bar_width, label=f"Temperature = {T}")

ax.set_ylabel("Probability")
ax.set_xticks(x)
ax.set_xticklabels(vocab.keys(), rotation=90)
ax.legend()

plt.tight_layout()
plt.savefig("tempareture-plot.pdf")
plt.show()

tempareture=1: 温度スケーリングを使用しないことと同じ

温度設定を高くすれば、生成されるテキストの多様性を高めることができるが、意味不明なテキストが生成されるケースも多くなる

In [ ]:
# top-kサンプリング
# 選択の対象となるトークンを最も高い上位k個のトークンに限定できる
# それ以外のトークンについては、それらの確率スコアをマスクすることで選択プロセスから除外できる（対象外の選択を負の無限大(-inf)でマスク）

top_k = 3
top_logits, top_pos = torch.topk(next_token_logits, top_k)
print(f"Top logits: {top_logits}")
print(f"Top positions: {top_pos}")

In [ ]:
# ロジットの大きさが上位3つに含まれないトークンのロジットを負の無限大（-inf）に設定
new_logits = torch.where(
    condition=next_token_logits < top_logits[-1], # 上位3つのロジットよりも小さいロジットを特定
    input=torch.tensor(float("-inf")), # 上位3つ以外を-infに設定
    other=next_token_logits # それ以外はそのまま
)
print(new_logits)

In [ ]:
# 次に来るトークンの確率に変換
topk_probas = torch.softmax(new_logits, dim=0)
print(topk_probas)

In [ ]:
# 多様性を高める新しいテキスト生成関数
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        if top_k is not None:
            topk_logits, _ = torch.topk(logits, top_k)
            min_val = topk_logits[:, -1]
            logits = torch.where(
                logits < min_val,
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )

        if temperature > 0.0:
            logits = logits / temperature
            logits = logits - logits.max(dim=-1, keepdim=True).values
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else:
            # 温度スケーリングが無効の場合は従来同様に貪欲なデコーディング
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)

        # シーケンス終了トークンが検出
        # 生成を早期終了
        if idx_next == eos_id:
            break

        idx = torch.cat((idx, idx_next), dim=1)

    return idx

In [ ]:
# 使用してみる
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to("cpu"),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)

print(f"Output text:\n {token_ids_to_text(token_ids, tokenizer)}")

## 5.4 PyTorchでモデルの重みの保存と読み込み

In [ ]:
# モデルパラメータの保存
# ".pth"はPyTorchファイルの慣例的な拡張子 -> 厳密にはどんな拡張子使っても良い
torch.save(model.state_dict(), "model.pth")

In [ ]:
# GPTModelインスタンスで保存したモデル情報の読み込み
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth", map_location=device))
model.eval()

AdamWのような適応型オプティマイザは、各モデルの重みに追加のパラメータを格納する

AdamWは過去のデータをもとに各モデルパラメータの学習率を動的に調整する

このようにしないと、オプティマイザがリセットされ、モデルの学習が最適でなくなったり、場合によっては上手く収束しなくなったりして、一貫性のあるテキストを生成する能力がそこなわれるためである

In [ ]:
# モデルとオプティマイザの情報を保存
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
},
    "model_and_optimizer.pth")

In [ ]:
# モデルとオプティマイザを保存した情報から読み込み
checkpoint = torch.load("model_and_optimizer.pth", map_location=device)
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train()

## 5.5 OpenAIから事前学習済みの重みを読み込む
- Linear層、Embedding層のweight属性に格納されている重みパラメータをOpenAIが公開している情報を使用して読み込む

In [ ]:
!pip install tensorflow>=2.15.0 tqdm>=4.66

In [ ]:
# バージョン確認
from importlib.metadata import version

print(f"Tensorflow version: {version("tensorflow")}")
print(f"tqdm version: {version("tqdm")}")

In [ ]:
# Copyright (c) Sebastian Raschka under Apache License 2.0 (see LICENSE.txt).
# Source for "Build a Large Language Model From Scratch"
#   - https://www.manning.com/books/build-a-large-language-model-from-scratch
# Code: https://github.com/rasbt/LLMs-from-scratch


import os

import requests
import json
import numpy as np
import tensorflow as tf
from tqdm import tqdm


def download_and_load_gpt2(model_size, models_dir):
    # Validate model size
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")

    # Define paths
    model_dir = os.path.join(models_dir, model_size)
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    backup_base_url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2"
    filenames = [
        "checkpoint", "encoder.json", "hparams.json",
        "model.ckpt.data-00000-of-00001", "model.ckpt.index",
        "model.ckpt.meta", "vocab.bpe"
    ]

    # Download files
    os.makedirs(model_dir, exist_ok=True)
    for filename in filenames:
        file_url = os.path.join(base_url, model_size, filename)
        backup_url = os.path.join(backup_base_url, model_size, filename)
        file_path = os.path.join(model_dir, filename)
        download_file(file_url, file_path, backup_url)

    # Load settings and params
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    settings = json.load(open(os.path.join(model_dir, "hparams.json"), "r", encoding="utf-8"))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

    return settings, params


def download_file(url, destination, backup_url=None):
    def _attempt_download(download_url):
        response = requests.get(download_url, stream=True, timeout=60)
        response.raise_for_status()

        file_size = int(response.headers.get("Content-Length", 0))

        # Check if file exists and has same size
        if os.path.exists(destination):
            file_size_local = os.path.getsize(destination)
            if file_size and file_size == file_size_local:
                print(f"File already exists and is up-to-date: {destination}")
                return True

        block_size = 1024  # 1 KB
        desc = os.path.basename(download_url)
        with tqdm(total=file_size, unit="iB", unit_scale=True, desc=desc) as progress_bar:
            with open(destination, "wb") as file:
                for chunk in response.iter_content(chunk_size=block_size):
                    if chunk:
                        file.write(chunk)
                        progress_bar.update(len(chunk))
        return True

    try:
        if _attempt_download(url):
            return
    except requests.exceptions.RequestException:
        if backup_url is not None:
            print(f"Primary URL ({url}) failed. Attempting backup URL: {backup_url}")
            try:
                if _attempt_download(backup_url):
                    return
            except requests.exceptions.RequestException:
                pass

        error_message = (
            f"Failed to download from both primary URL ({url})"
            f"{' and backup URL (' + backup_url + ')' if backup_url else ''}."
            "\nCheck your internet connection or the file availability.\n"
            "For help, visit: https://github.com/rasbt/LLMs-from-scratch/discussions/273"
        )
        print(error_message)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Alternative way using `requests`
"""
def download_file(url, destination):
    # Send a GET request to download the file in streaming mode
    response = requests.get(url, stream=True)

    # Get the total file size from headers, defaulting to 0 if not present
    file_size = int(response.headers.get("content-length", 0))

    # Check if file exists and has the same size
    if os.path.exists(destination):
        file_size_local = os.path.getsize(destination)
        if file_size == file_size_local:
            print(f"File already exists and is up-to-date: {destination}")
            return

    # Define the block size for reading the file
    block_size = 1024  # 1 Kilobyte

    # Initialize the progress bar with total file size
    progress_bar_description = url.split("/")[-1]  # Extract filename from URL
    with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
        # Open the destination file in binary write mode
        with open(destination, "wb") as file:
            # Iterate over the file data in chunks
            for chunk in response.iter_content(block_size):
                progress_bar.update(len(chunk))  # Update progress bar
                file.write(chunk)  # Write the chunk to the file
"""


def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    # Initialize parameters dictionary with empty blocks for each layer
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Iterate over each variable in the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable and remove singleton dimensions
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

        # Process the variable name to extract relevant parts
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Identify the target dictionary for the variable
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Recursively access or create nested dictionaries
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Assign the variable array to the last key
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return params

In [ ]:
# # GitHubのリポジトリから読み込む
# import urllib.request

# url = ("https://github.com/rasbt/LLMs-from-scratch/blob/main/ch05/01_main-chapter-code/gpt_download.py")
# filename = url.split("/")[-1]
# urllib.request.urlretrieve(url, filename)

In [ ]:
settings, params = download_and_load_gpt2(
    model_size="124M",
    models_dir="gpt2"
)

In [ ]:
# settingsとparamsの内容を確認
print(f"Settings: {settings}")
print(f"Parameter dectionary keys: {params.keys()}")

In [ ]:
# トークンの埋め込み層の重みを表示
print(params["wte"])
print(f"Token embedding weight tensor dimenstions: {params["wte"].shape}")

GPT-2モデルアーキテクチャのパラメータ数の違い

基本的なアーキテクチャは同じ

個々のコンポーネント（Attentionヘッド、Transformerブロックなど）の繰り返し回数と埋め込みサイズが異なる

In [ ]:
# モデルの設定
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# モデルパラメータの設定を更新
model_name = "gpt2-small (124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

# OpenAIのモデルはコンテキストの長さが1,024でバイアスを使っている
# 一貫性を保つために、これらのパラメータ内容を更新して、設定を一致させる
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval()

In [ ]:
# GPTModelインスタンスは事前学習のためにランダムな重みで初期化される
# OpenAIのモデルの重みを使うための最後のステップは、このランダムな重みをparamsディクショナリに読み込んだ重みで上書きする
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismath. Left: {left.shape}, Right: {right.shape}")
    # 形状が同じであれば訓練可能なPyTorchパラメータとして、rightテンソルを返す
    return torch.nn.Parameter(torch.tensor(right))

In [ ]:
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].attn.W_query.weight = assign(
            gpt.trf_blocks[b].attn.W_query.weight, q_w.T)
        gpt.trf_blocks[b].attn.W_key.weight = assign(
            gpt.trf_blocks[b].attn.W_key.weight, k_w.T)
        gpt.trf_blocks[b].attn.W_value.weight = assign(
            gpt.trf_blocks[b].attn.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].attn.W_query.bias = assign(
            gpt.trf_blocks[b].attn.W_query.bias, q_b)
        gpt.trf_blocks[b].attn.W_key.bias = assign(
            gpt.trf_blocks[b].attn.W_key.bias, k_b)
        gpt.trf_blocks[b].attn.W_value.bias = assign(
            gpt.trf_blocks[b].attn.W_value.bias, v_b)

        gpt.trf_blocks[b].attn.out_proj.weight = assign(
            gpt.trf_blocks[b].attn.out_proj.weight,
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].attn.out_proj.bias = assign(
            gpt.trf_blocks[b].attn.out_proj.bias,
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight,
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias,
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight,
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias,
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale,
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift,
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale,
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift,
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])


load_weights_into_gpt(gpt, params)
gpt.to(device);

load_weight_info_gpt()関数では、OpenAIの実装とGPTModelの実装の重みを慎重にマッチさせている。

In [ ]:
# OpenAIモデルの重みをGPTModelのインスタンスgptに読み込み
load_weights_into_gpt(gpt, params)
gpt.to(device)

In [ ]:
gpt.trf_blocks[0].attn

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)
print(f"Output text:\n{token_ids_to_text(token_ids, tokenizer)}")

In [ ]:
# 練習問題 5-5
# 事前学習済みの重みを適用したモデルを使用して損失を計算

# calc_loss_loader()を訓練データセットと検証データセットに適用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpt.to(device) # GPU搭載であればLLMの訓練をGPUで行う

with torch.no_grad():
    # まだ訓練していないため、効率化のために勾配の追跡を無効にする
    train_loss = calc_loss_loader(train_loader, gpt, device)
    val_loss = calc_loss_loader(val_loader, gpt, device)

print(f"Training loss: {train_loss}")
print(f"Validation loss: {val_loss}")

In [ ]:
# 練習問題 5-6
# 1.5Bのモデルで生成したテキストの内容確認
# モデルの設定
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# モデルパラメータの設定を更新
model_name = "gpt2-xl (1558M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

# OpenAIのモデルはコンテキストの長さが1,024でバイアスを使っている
# 一貫性を保つために、これらのパラメータ内容を更新して、設定を一致させる
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt_1558m = GPTModel(NEW_CONFIG)
gpt_1558m.eval()

In [ ]:
# OpenAI側
settings, params = download_and_load_gpt2(
    model_size="1558M",
    models_dir="gpt2"
)

In [ ]:
load_weights_into_gpt(gpt_1558m, params)
gpt_1558m.to(device);

In [ ]:
# テキスト生成
torch.manual_seed(123)

token_ids = generate(
    model=gpt_1558m,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)
print(f"Output text:\n{token_ids_to_text(token_ids, tokenizer)}")

In [ ]:
# 練習問題 5-5
# 事前学習済みの重みを適用したモデルを使用して損失を計算

# calc_loss_loader()を訓練データセットと検証データセットに適用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpt_1558m.to(device) # GPU搭載であればLLMの訓練をGPUで行う

with torch.no_grad():
    # まだ訓練していないため、効率化のために勾配の追跡を無効にする
    train_loss = calc_loss_loader(train_loader, gpt_1558m, device)
    val_loss = calc_loss_loader(val_loader, gpt_1558m, device)

print(f"Training loss: {train_loss}")
print(f"Validation loss: {val_loss}")